#### **Importing Libraries**

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

##### **Extracting Data**

In [2]:
df= pd.read_csv("C:/Users/DELL/Downloads/messy_large_employee_data.csv")
df.head()

,Hired_Date,Leave_Date,Gender,Marital_Status,Job_Title,Salary,Department,Job_Category
0,2011-06-27,NaN,f.,Widowed,Data Anlyst,117744.0,IT,Executive
1,2000-05-06,2003-10-05,f.,Divorced,Business Analyst,119672.0,Human Resources,Manager
2,2010-01-16,2013-12-09,mALE,Widowed,Business Analyst,38849.0,Marketing,NaN
3,2009-10-17,NaN,M.,single,Sr. Data Analyst,57301.0,Sales,Executive
4,2000-02-15,2012-08-18,mALE,Widowed,Data Engineer,47959.0,HR,Management


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 8 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Hired_Date      1200000 non-null  object 
 1   Leave_Date      360211 non-null   object 
 2   Gender          1200000 non-null  object 
 3   Marital_Status  1050352 non-null  object 
 4   Job_Title       1200000 non-null  object 
 5   Salary          1139710 non-null  float64
 6   Department      1200000 non-null  object 
 7   Job_Category    1049784 non-null  object 
dtypes: float64(1), object(7)
memory usage: 73.2+ MB


In [4]:
df.shape

(1200000, 8)

#### Data Cleaning

In [5]:
df['Gender'].unique()

array(['f.', 'mALE', 'M.', 'FEMALE', 'F', 'Female', 'M', 'Male'],
      dtype=object)

In [6]:
df['Job_Category'].unique()

array(['Executive', 'Manager', nan, 'Management', 'Tech', 'Technical',
       'Admin', 'Administration'], dtype=object)

In [7]:
df['Marital_Status'].unique()

array(['Widowed', 'Divorced', 'single', 'divorced', 'Single', nan,
       'Married', 'MARRIED'], dtype=object)

In [8]:
df['Job_Title'].unique()

array(['Data Anlyst', 'Business Analyst', 'Sr. Data Analyst',
       'Data Engineer', 'Junior Analyst', 'BI Developer',
       'Sr Data Scientist', 'Data Analyst', 'Data Scientist',
       'Jr. Data Analyst'], dtype=object)

In [9]:
df['Department'].unique()

array(['IT', 'Human Resources', 'Marketing', 'Sales', 'HR', 'SALES',
       'Information Technology', 'FIN', 'Finance'], dtype=object)

##### **Fetching & Removal of Duplicates**

* Fetching Duplicates

In [10]:
df.duplicated().sum()

6

* Removing Duplicates

In [11]:
df.drop_duplicates(inplace=True)

* Converting HiredDate and LeaveDate from Object Type to DateTime

In [12]:
df['Hired_Date']= pd.to_datetime(df['Hired_Date'])
df['Leave_Date']= pd.to_datetime(df['Leave_Date'])

In [13]:
df.dtypes

Hired_Date        datetime64[ns]
Leave_Date        datetime64[ns]
Gender                    object
Marital_Status            object
Job_Title                 object
Salary                   float64
Department                object
Job_Category              object
dtype: object

* Creating RowID for Fast Reterival on the MSSQL Database

In [14]:
df= df.sort_values(by=['Hired_Date'],ascending=True)
df['EmpID']= range(1,len(df)+1)

In [15]:
df.head()

,Hired_Date,Leave_Date,Gender,Marital_Status,Job_Title,Salary,Department,Job_Category,EmpID
996480,2000-01-01,NaT,f.,Single,Sr. Data Analyst,106044.0,Human Resources,Admin,1
907377,2000-01-01,2013-04-11,f.,MARRIED,Data Engineer,86555.0,Finance,Manager,2
1188337,2000-01-01,2008-02-02,Male,MARRIED,BI Developer,79929.0,FIN,Administration,3
142706,2000-01-01,2005-09-06,Female,NaN,Junior Analyst,101615.0,SALES,Tech,4
907545,2000-01-01,2009-08-12,Male,divorced,Junior Analyst,37819.0,IT,Tech,5


* Reordering Colummn

In [16]:
df= df[['EmpID'] + [col for col in df.columns if col!='EmpID']]

In [17]:
df.head()

,EmpID,Hired_Date,Leave_Date,Gender,Marital_Status,Job_Title,Salary,Department,Job_Category
996480,1,2000-01-01,NaT,f.,Single,Sr. Data Analyst,106044.0,Human Resources,Admin
907377,2,2000-01-01,2013-04-11,f.,MARRIED,Data Engineer,86555.0,Finance,Manager
1188337,3,2000-01-01,2008-02-02,Male,MARRIED,BI Developer,79929.0,FIN,Administration
142706,4,2000-01-01,2005-09-06,Female,NaN,Junior Analyst,101615.0,SALES,Tech
907545,5,2000-01-01,2009-08-12,Male,divorced,Junior Analyst,37819.0,IT,Tech


* Transforming Gender Column

In [18]:
df['Gender']= df['Gender'].replace({'F': 'Female','f': 'Female','f.':'Female','FEMALE':'Female'})
df['Gender']= df['Gender'].replace({'M.': 'Male','M':'Male','mALE':'Male'})
df['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [19]:
df['Marital_Status'].value_counts()

Marital_Status
Married     150742
MARRIED     150195
divorced    150132
Widowed     150084
Divorced    149989
Single      149735
single      149469
Name: count, dtype: int64

* Transforming Marital_Status Column

In [20]:
df['Marital_Status']= df['Marital_Status'].replace({'single': 'Single','MARRIED':'Married'})
df['Marital_Status']= df['Marital_Status'].replace({'divorced': 'Divorced','MARRIED':'Married'})
df['Marital_Status']= df['Marital_Status'].replace({None: 'Single'})

In [21]:
df['Marital_Status'].unique()

array(['Single', 'Married', 'Divorced', 'Widowed'], dtype=object)

* Transforming Salary Column

In [22]:
df['Salary'].unique()

array([106044.,  86555.,  79929., ...,  78282.,  57665.,  53231.])

* Replacing Null Values in Salary Column with Average Salary

In [23]:
df['Salary']= df['Salary'].fillna(df['Salary'].mean())

In [24]:
df['Salary'].unique()

array([106044.,  86555.,  79929., ...,  78282.,  57665.,  53231.])

* Transforming Department Column

In [25]:
df['Department'].unique()

array(['Human Resources', 'Finance', 'FIN', 'SALES', 'IT', 'HR', 'Sales',
       'Marketing', 'Information Technology'], dtype=object)

In [26]:
df['Department']= df['Department'].replace({'FIN':'Finance','IT':'Information Technology'})
df['Department']= df['Department'].replace({'SALES':'Sales','HR':'Human Resources'})

In [27]:
df['Department'].unique()

array(['Human Resources', 'Finance', 'Sales', 'Information Technology',
       'Marketing'], dtype=object)

* Transforming Job Category Column

In [28]:
df['Job_Category'].unique()

array(['Admin', 'Manager', 'Administration', 'Tech', 'Management',
       'Technical', 'Executive', nan], dtype=object)

In [29]:
df['Job_Category'].value_counts()

Job_Category
Management        150402
Administration    150291
Technical         150128
Executive         149899
Tech              149741
Manager           149711
Admin             149606
Name: count, dtype: int64

In [30]:
df['Job_Category']= df['Job_Category'].replace({'Admin':'Administration','Tech':'Technical'})
df['Job_Category']= df['Job_Category'].replace({None:'Management','Manager':'Management'})

In [31]:
df['Job_Category'].unique()

array(['Administration', 'Management', 'Technical', 'Executive'],
      dtype=object)

* Transforming Job_Title

In [32]:
df['Job_Title'].unique()

array(['Sr. Data Analyst', 'Data Engineer', 'BI Developer',
       'Junior Analyst', 'Data Anlyst', 'Data Scientist',
       'Sr Data Scientist', 'Jr. Data Analyst', 'Data Analyst',
       'Business Analyst'], dtype=object)

In [33]:
df['Job_Title']= df['Job_Title'].replace({'Junior Analyst':'Jr. Data Analyst','Data Anlyst':'Data Analyst'})
df['Job_Title']= df['Job_Title'].replace({'Sr Data Scientist':'Sr. Data Scientist'})
df['Job_Title'].unique()

array(['Sr. Data Analyst', 'Data Engineer', 'BI Developer',
       'Jr. Data Analyst', 'Data Analyst', 'Data Scientist',
       'Sr. Data Scientist', 'Business Analyst'], dtype=object)

In [34]:
df.head()

,EmpID,Hired_Date,Leave_Date,Gender,Marital_Status,Job_Title,Salary,Department,Job_Category
996480,1,2000-01-01,NaT,Female,Single,Sr. Data Analyst,106044.0,Human Resources,Administration
907377,2,2000-01-01,2013-04-11,Female,Married,Data Engineer,86555.0,Finance,Management
1188337,3,2000-01-01,2008-02-02,Male,Married,BI Developer,79929.0,Finance,Administration
142706,4,2000-01-01,2005-09-06,Female,Single,Jr. Data Analyst,101615.0,Sales,Technical
907545,5,2000-01-01,2009-08-12,Male,Divorced,Jr. Data Analyst,37819.0,Information Technology,Technical


In [35]:
df.shape

(1199994, 9)

##### **Creating JobTitleID, DepartmentID & JobCategoryID Columns**

* JobTitle ID

In [36]:
df['Job_Title'].unique()

array(['Sr. Data Analyst', 'Data Engineer', 'BI Developer',
       'Jr. Data Analyst', 'Data Analyst', 'Data Scientist',
       'Sr. Data Scientist', 'Business Analyst'], dtype=object)

In [37]:
condition= [ (df['Job_Title']=='Jr. Data Analyst'),
             (df['Job_Title']=='Data Analyst'),
             (df['Job_Title']=='Data Scientist'),
             (df['Job_Title']=='Data Engineer'),
             (df['Job_Title']=='Business Analyst'),
             (df['Job_Title']=='BI Developer'),
             (df['Job_Title']=='Sr. Data Analyst'),
             (df['Job_Title']=='Sr. Data Scientist')]
output= [1,2,3,4,5,6,7,8]
df['JobTitleID']= np.select(condition,output)

* DepartmentID

In [38]:
df['Department'].unique()

array(['Human Resources', 'Finance', 'Sales', 'Information Technology',
       'Marketing'], dtype=object)

In [39]:
condition= [ (df['Department']=='Human Resources'),
             (df['Department']=='Finance'),
             (df['Department']=='Sales'),
             (df['Department']=='Information Technology'),
             (df['Department']=='Marketing')]
output= [1,2,3,4,5]
df['DepartmentID']= np.select(condition,output)

* JobCategoryID

In [40]:
df['Job_Category'].unique()

array(['Administration', 'Management', 'Technical', 'Executive'],
      dtype=object)

In [41]:
condition= [ (df['Job_Category']=='Administration'),
             (df['Job_Category']=='Management'),
             (df['Job_Category']=='Technical'),
             (df['Job_Category']=='Executive')]
output= [1,2,3,4]
df['JobCategoryID']= np.select(condition,output)

#### **Reordering Columns for Readability**

In [42]:
new_column_order= ['EmpID','Hired_Date','Leave_Date','Gender','Marital_Status',
                   'JobTitleID','Job_Title','Salary','DepartmentID','Department',
                   'JobCategoryID','Job_Category']
df= df[new_column_order]
df.head()

,EmpID,Hired_Date,Leave_Date,Gender,Marital_Status,JobTitleID,Job_Title,Salary,DepartmentID,Department,JobCategoryID,Job_Category
996480,1,2000-01-01,NaT,Female,Single,7,Sr. Data Analyst,106044.0,1,Human Resources,1,Administration
907377,2,2000-01-01,2013-04-11,Female,Married,4,Data Engineer,86555.0,2,Finance,2,Management
1188337,3,2000-01-01,2008-02-02,Male,Married,6,BI Developer,79929.0,2,Finance,1,Administration
142706,4,2000-01-01,2005-09-06,Female,Single,1,Jr. Data Analyst,101615.0,3,Sales,3,Technical
907545,5,2000-01-01,2009-08-12,Male,Divorced,1,Jr. Data Analyst,37819.0,4,Information Technology,3,Technical


#### **Creating Connection to MSSQL Database**

In [43]:
Server= 'DESKTOP-ULUKUD4'
Database= 'HRdb'
Driver= 'ODBC Driver 17 for SQL Server'
Database_Connection= f'mssql://{Server}/{Database}?driver={Driver}'
engine= create_engine(Database_Connection)
connection= engine.connect()

##### **Load the Transformed data to MSSQL Database**

In [44]:
df.to_sql('employee',if_exists='replace',con=engine,index=False)

90

#### **Save to CSV File**

In [45]:
df.to_csv('employee.csv',index=False)